In [ ]:
# P Naidoo

In [ ]:
!pip install scipy
!pip install pandas
!pip install shutil
!pip install opencv-python
!pip install tabulate
!pip install sympy
!pip install scikit-image
!apt update && apt install -y libsm6 libxext6
!apt apt-get install -y libxrender-dev
#!apt-get update
!apt-get install ffmpeg libsm6 libxext6  -y

import os
import csv
import shutil
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow import keras

from IPython.display import clear_output

import random

import cv2

import pickle

LOCAL_DATA_PATH = '/tf/notebooks/Data/echonet frames split for model training' #'/tf/notebooks/Data/echonet'

MODEL_PATH = '/tf/notebooks/Project with Beth/Trained Models/Model training on ECHONET with Tversky loss'

SAVE_FOLDER = f'Prediction Results'



# Settings

In [ ]:

OUTPUT_CHANNELS = 1 # no of classes
BATCH_SIZE = 8      # 8 for GPU Server
BUFFER_SIZE = 1000
EPOCHS = 100         # 100 for GPU Server
INPUT_IMAGE_SIZE_PIXELS = 512
INPUT_SHAPE_IMAGE = [512, 512, 1]
LEARNING_RATE=0.00001
VAL_SUBSPLITS = 5

NUM_OF_EXPERIMENTS = 5      #How many times to run each task

EARLY_STOPPING_PATIENCE = 10 # 3 is the default value in tensorflow

SEED = 24

START_PERCENTAGE = 4        #example start at 4% since the initial model was trained at 4% but do not train the first 4%
INCREMENT_PERCENTAGE = 1    #increment every k%
STOP_PERCENTAGE = 50        #stop at 30% of training data

# Functions for reading images, displaying predictions, and callback

In [ ]:
def parse_image(img_path: str) -> dict:
    """Load an image and its annotation (mask) and return a dictionary.

    img_path : str : Image (not the mask) location.
    dict: Dictionary mapping an image and its annotation.
    """
    image = tf.io.read_file(img_path)
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.convert_image_dtype(image, tf.uint8)

    # For one Image path:
    # .../dataset/images/training/im_00000001.jpg
    # Its corresponding annotation path is:
    # .../dataset/annotations/training/im_00000001.png
    mask_path = tf.strings.regex_replace(img_path, "images", "annotations_binary")
    mask_path = tf.strings.regex_replace(mask_path, "jpg", "png")
    mask = tf.io.read_file(mask_path)
    # The masks contain a class index for each pixels
    mask = tf.image.decode_png(mask, channels=1)
    # In scene parsing, "not labeled" = 255
    # But it will mess up with our N_CLASS = 150
    # Since 255 means the 255th class, which doesn't exist
    #mask = tf.where(mask == 255, np.dtype('uint8').type(0), mask)
    # Note that we have to convert the new value (0)
    # With the same dtype than the tensor itself    

    return {'image': image, 'segmentation_mask': mask}

In [ ]:
def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0  
  return input_image, input_mask

In [ ]:
def load_image(datapoint):
  input_image = tf.image.resize(datapoint['image'], (INPUT_IMAGE_SIZE_PIXELS, INPUT_IMAGE_SIZE_PIXELS))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (INPUT_IMAGE_SIZE_PIXELS, INPUT_IMAGE_SIZE_PIXELS))

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [ ]:
#Visualize an image example and its corresponding 
#mask from the dataset.

def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()


def display1(display_list):
  plt.figure(figsize=(15, 15))

  title = ['True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(display_list[i])
    plt.axis('off')
  plt.show()

In [ ]:
def create_mask(pred_mask):
  #pred_mask = tf.argmax(pred_mask, axis=-1)
  #pred_mask = pred_mask[..., tf.newaxis]
  pred_mask = tf.greater(pred_mask, 0.5)
  pred_mask = tf.dtypes.cast(pred_mask, tf.float32)
  #pred_mask = pred_mask[..., tf.newaxis]
  pred_mask = pred_mask[0]
  return pred_mask

In [ ]:
def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      display([image[0], mask[0], create_mask(pred_mask)])
  else:
    display([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])

In [ ]:
def create_mask2(pred_mask):
  #pred_mask = tf.argmax(pred_mask, axis=-1)
  #pred_mask = pred_mask[..., tf.newaxis]
  pred_mask = tf.greater(pred_mask, 0.5)
  pred_mask = tf.dtypes.cast(pred_mask, tf.float32)
  #pred_mask = pred_mask[..., tf.newaxis]
  pred_mask = pred_mask[1]
  return pred_mask

def show_predictions_manual(image, mask, predicted): 
    display([image[0], mask[0], create_mask(predicted)])
    
def show_predictions_manual2(image, mask, predicted): 
    display([image[0], mask[0], create_mask2(predicted)])
  

In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    show_predictions()
    print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

In [ ]:
#How to call above functions:

#for images, masks in train.take(2):
#  sample_image, sample_mask = images, masks
#  display([sample_image, sample_mask])


#show_predictions(test_data, 3)

# SAVE FILES

In [ ]:
def save_experiment_to_file(directory, filename, index, data):
    
    new_file_name = f'{filename}.pkl'
    if not(index is None):
        new_file_name = f'{filename}_{index}.pkl'
        
    file_name_with_path = os.path.join(directory, new_file_name)
    with open(file_name_with_path, 'wb') as filehandle:
        pickle.dump(data, filehandle)
    
def load_experiment_from_file(directory, filename, index):
    
    new_file_name = f'{filename}.pkl'
    if not(index is None):
        new_file_name = f'{filename}_{index}.pkl'
    
    file_name_with_path = os.path.join(directory, new_file_name)
    with open(file_name_with_path, 'rb') as filehandle:
        return pickle.load(filehandle)

# Metric Functions

In [ ]:
from scipy.spatial.distance import dice

def compute_Dice_coefficient(original_mask, predicted_mask):
    '''
            2 x Intersection
    DICE = -------------------
           Union + Intersection
           
    This could be seen as:
    
                2 x TP
    DICE = -----------------
           (FP + TP + FN) + TP
    '''
    
    #a = original_mask.ravel()
    #b = predicted_mask.ravel()
    
    #a = original_mask.flatten()
    #b = predicted_mask.flatten()    
    
    #count1 = (a == 1).sum()
    #count2 = (b == 1).sum()
    
    #count1 = np.count_nonzero(original_mask)
    #count2 = np.count_nonzero(predicted_mask)
    
    a = np.array(original_mask, dtype=np.bool)
    a = np.atleast_1d(a)
    a = tf.reshape(a, [-1])
    
    b = np.array(predicted_mask, dtype=np.bool)
    b = np.atleast_1d(b)
    b = tf.reshape(b, [-1])    
    
    #dice original code, first attempt
    #intersection = np.count_nonzero(a & b)    
    #union = np.count_nonzero(a) + np.count_nonzero(b)
    #dc = (2. * intersection)/float(union) # + intersection - intersection
    
    #dice computation, second attempt using scipy function
    #dc = dice(a, b)
    
    #dice computation, third attempt
    dc = np.sum(predicted_mask[original_mask==1])*2.0 / (np.sum(predicted_mask) + np.sum(original_mask))
    
    #All three above produce the same output. The second attemp needs to be subtracted from 1.
    
    return dc

def compute_IoU(original_mask, predicted_mask):    
    '''
            Intersection
    IoU = -------------------
                Union
           
    This could be seen as:
    
                 TP
    IoU = -----------------
           (FP + TP + FN)
    
    '''
    
    a = np.array(original_mask, dtype=np.bool)
    a = np.atleast_1d(a)
    a = tf.reshape(a, [-1])
    
    b = np.array(predicted_mask, dtype=np.bool)
    b = np.atleast_1d(b)
    b = tf.reshape(b, [-1])   
    
    #IoU Attempt 1
    #intersection = np.count_nonzero(a & b)    
    #union = (np.count_nonzero(a) + np.count_nonzero(b)) - intersection      
    #iou = intersection/union
    
    #IoU attempt 2
    intersection = np.sum(predicted_mask[original_mask==1])
    iou = (intersection) / ((np.sum(predicted_mask) + np.sum(original_mask)) - intersection)
    
    #Attempt 1 and 2 above produce the same output. i.e. both works.
    
    return iou

In [ ]:
from scipy.spatial.distance import directed_hausdorff

def compute_Hausdorff_distance(original_mask, predicted_mask):
    '''
    We're going to use the Distance Map(Distance Transform) method of computing the Hausdorf distance.
    A python library Scipy has a function that can do this for us called distance_transform_edt.
    After computing the distance map for mask2 i.e DM2, we need to overlap the boundary of mask1 onto 
    DM2. The take the maximum value in DM2 where mask1 overlaps. 
    This maximum value will be the Hausdorf distance d(mask1, mask2).
    Then we need to find the Hausdorf distance d(mask2, mask1).
    The final distance will be the max(d(mask1, mask2), d(mask2, mask1))
    
    However, first lets try to do this using scipy.spatial.distance.directed_hausdorf
    
    Note: for distance, should we use Euclidean or Manhattan etc?
    
    https://en.wikipedia.org/wiki/Distance_transform
    
    https://cs.stackexchange.com/questions/117989/hausdorff-distance-between-two-binary-images-according-to-distance-maps
    
    https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.ndimage.morphology.distance_transform_edt.html
    
    Paper: AN IMAGE ALGORITHM FOR COMPUTING THE SDORFF DISTANCE EFFICIENTLY IN LINEAR TIME
    Paper: A Linear Time Algorithm of Computing Hausdorff Distance for Content-based Image Analysis    
    
    '''
    
    hd1 = directed_hausdorff(original_mask, predicted_mask)[0]
    hd2 = directed_hausdorff(predicted_mask, original_mask)[0]
    
    return max(hd1, hd2)    
    #return 0
   

In [ ]:

# https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient
# https://en.wikipedia.org/wiki/Jaccard_index


In [ ]:
def get_average_metrics(model, testing_data, num_test_examples, uncertainty = False, dropout = False):
    
    sum_dice=0
    #sum_IoU=0
    sum_dH=0
    c=0
    
    uncertainty = 0;
    
    all_distances={}
    
    all_uncertainties={}
    
    for image, mask in testing_data.take(num_test_examples):
    
        predicted = None
        if(dropout):
            predicted = model(image, training = True).numpy()
        else:
            predicted = model.predict(image)
            
        predicted_mask = create_mask(predicted)  

        predicted_mask = predicted_mask.numpy()
        predicted_mask = predicted_mask[:, :, 0] #drop the third dimension here since its 512x512x1

        ground_truth_mask = mask[0].numpy()
        ground_truth_mask = ground_truth_mask[:, :, 0]   

        DC = compute_Dice_coefficient(ground_truth_mask, predicted_mask)
        #IoU = compute_IoU(ground_truth_mask, predicted_mask)
        dH = compute_Hausdorff_distance(ground_truth_mask, predicted_mask)
        
        if(uncertainty):
            uncertainty = compute_uncertainty(predicted[0])
            all_uncertainties[c] = uncertainty
        #show_predictions_manual(image, mask, predicted)        

        #show top 5
        #if(c<5):
        #    print(f'Dice coefficient: {DC}')
        #    print(f'Loss: {1-DC}')
        #    print(f'IoU: {IoU}')
        #    print(f'Hausdorff Distance: {dH}')
        #    print()
            #print(f'Jaccard Index:{J_Index}')        
            
        all_distances[c] = dH        
        sum_dice += DC
        #sum_IoU += IoU
        sum_dH += dH
        c += 1
        
    avg_dice = (sum_dice)/num_test_examples
    #avg_IoU = (sum_IoU)/num_test_examples
    avg_dH = (sum_dH)/num_test_examples
        

    return avg_dice, avg_dH, all_distances, all_uncertainties

In [ ]:
def get_average_metrics_with_masks(model, testing_data, num_test_examples, mute = True):
    
    sum_dice=0
    sum_IoU=0
    sum_dH=0
    c=0
    
    uncertainty = 0;
    
    all_distances={}
    
    all_uncertainties={}
    
    all_predicted_masks = {}
    
    all_ground_truth_masks = {}
    
    for image, mask in testing_data.take(num_test_examples):
    
        predicted = model.predict(image)
        predicted_mask = create_mask(predicted)  

        predicted_mask = predicted_mask.numpy()
        predicted_mask = predicted_mask[:, :, 0] #drop the third dimension here since its 512x512x1

        ground_truth_mask = mask[0].numpy()
        ground_truth_mask = ground_truth_mask[:, :, 0]   

        DC = compute_Dice_coefficient(ground_truth_mask, predicted_mask)
        IoU = compute_IoU(ground_truth_mask, predicted_mask)
        dH = compute_Hausdorff_distance(ground_truth_mask, predicted_mask)
        #DC = 0
        #IoU = 0
        #dH = 0
        
        uncertainty = compute_uncertainty(predicted[0])
        #show_predictions_manual(image, mask, predicted)        

        #show top 5
        #if(c<5):
        #    print(f'Dice coefficient: {DC}')
        #    print(f'Loss: {1-DC}')
        #    print(f'IoU: {IoU}')
        #    print(f'Hausdorff Distance: {dH}')
        #    print()
            #print(f'Jaccard Index:{J_Index}')
            
        all_predicted_masks[c] = predicted_mask
        all_ground_truth_masks[c] = ground_truth_mask
            
        all_distances[c] = dH
        all_uncertainties[c] = uncertainty
        sum_dice += DC
        sum_IoU += IoU
        sum_dH += dH
        c += 1
        
    avg_dice = (sum_dice)/num_test_examples
    avg_IoU = (sum_IoU)/num_test_examples
    avg_dH = (sum_dH)/num_test_examples
    
    if mute == False:
        print()
        print(f'The average Dice is {avg_dice}')
        print(f'The average IoU is {avg_IoU}')
        print(f'The average Hausdorff Distance is {avg_dH}')
        print()

    return avg_dice, avg_IoU, avg_dH, all_distances, all_uncertainties, all_predicted_masks, all_ground_truth_masks

# Prepare Files

In [ ]:
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)


# callback_early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=EARLY_STOPPING_PATIENCE, verbose=1, restore_best_weights=True)

# shape = shape=(2, 2)
# initializer = tf.initializers.he_normal()
# var = tf.Variable(initializer(shape=shape))


In [ ]:
def get_data(files):
    
    imgs = tf.data.Dataset.from_tensor_slices(files)
    parse_set = imgs.map(parse_image, num_parallel_calls=tf.data.AUTOTUNE)
    load_set = parse_set.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    data = load_set.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
    
    return data
        
def get_Hausdorff_distance(predicted_masks, ground_truth_masks):
    
    num_examples = len(predicted_masks)
    all_dHs = {}
    sum_dH = 0    
    
    for i in range(num_examples):
        ground_truth_mask = ground_truth_masks[i]
        predicted_mask = predicted_masks[i]
        dH = compute_Hausdorff_distance(ground_truth_mask, predicted_mask)
        sum_dH+= dH
        all_dHs[i] = dH
        
    avg_dH = (sum_dH)/num_examples
    
    return avg_dH, all_dHs

def get_Dice_Coefficient(predicted_masks, ground_truth_masks):
    
    num_examples = len(predicted_masks)
    all_dice = {}
    sum_dice = 0    
    
    for i in range(num_examples):
        ground_truth_mask = ground_truth_masks[i]
        predicted_mask = predicted_masks[i]
        DC = compute_Dice_coefficient(ground_truth_mask, predicted_mask)
        sum_dice+= DC
        all_dice[i] = DC
        
    avg_dice = (sum_dice)/num_examples
    
    return avg_dice, all_dice
        
def make_predictions(model, image_files):     
    
    c=0    
    
    all_predicted_masks = {}
    
    all_ground_truth_masks = {}
    
    all_probabilities = {}
    
    for filename in image_files:
        
        image = tf.io.read_file(filename)
        if ('jpg' in filename):
            image = tf.image.decode_jpeg(image, channels=1)
        else:
            image = tf.image.decode_png(image, channels=1)
        image = tf.image.convert_image_dtype(image, tf.uint8)
        
        mask_path = tf.strings.regex_replace(filename, "images", "annotations_binary")
        mask_path = tf.strings.regex_replace(mask_path, ".jpg", ".png")
        mask = tf.io.read_file(mask_path)
        # The masks contain a class index for each pixels
        mask = tf.image.decode_png(mask, channels=1)        

        datapoint = {'image': image, 'segmentation_mask': mask}
        
        input_image = tf.image.resize(datapoint['image'], (INPUT_IMAGE_SIZE_PIXELS, INPUT_IMAGE_SIZE_PIXELS))
        input_mask = tf.image.resize(datapoint['segmentation_mask'], (INPUT_IMAGE_SIZE_PIXELS, INPUT_IMAGE_SIZE_PIXELS))  
        input_image, input_mask = normalize(input_image, input_mask)
        
        image = input_image
        mask = input_mask
        
        image_reshape = tf.expand_dims(image,0)        
        predicted = model.predict(image_reshape)
        predicted_mask = create_mask(predicted)  

        predicted_mask = predicted_mask.numpy()        
        predicted_mask = predicted_mask[:, :, 0] #drop the third dimension here since its 512x512x1

        ground_truth_mask = mask.numpy()
        ground_truth_mask = ground_truth_mask[:, :, 0]           
        
        all_predicted_masks[c] = predicted_mask
        all_ground_truth_masks[c] = ground_truth_mask        
        all_probabilities[c] = predicted[0]       
        
        c += 1
    

    return all_predicted_masks, all_ground_truth_masks, all_probabilities




In [ ]:
def get_rgb_image(predicted_mask):

    img_temp_np = np.zeros((predicted_mask.shape[0],predicted_mask.shape[1],3))


    for i in range(predicted_mask.shape[0]):
         for j in range(predicted_mask.shape[1]):
             if predicted_mask[i,j] == 1:
                img_temp_np[i,j] = (int(255),int(255),int(255))
             else:
                img_temp_np[i,j] = (int(0),int(0),int(0))

    img = img_temp_np.astype(np.uint8)
    
    return img

def get_binary_image(mask):

    img_temp_np = np.zeros((mask.shape[0],mask.shape[1]))


    for i in range(mask.shape[0]):
         for j in range(mask.shape[1]):
            #if np.array_equal(mask[i,j], np.asarray((255,255,255))):             
            if mask[i,j][0] > 0:            
                img_temp_np[i,j] = 1
            else:
                img_temp_np[i,j] = 0

    img = img_temp_np.astype(np.uint8)
    
    return img

def draw_poly_on_image(img, coordinates, color, use_weight=True):
    pts = np.array(coordinates, np.int32)
    pts = pts.reshape((-1,1,2))
    
    weight = 1
    if (use_weight):
        weight = 2
        
    img = cv2.polylines(img, [pts], True, color, thickness=weight)
    return img

def draw_poly_on_image_weight(img, coordinates, color, weight=1):
    pts = np.array(coordinates, np.int32)
    pts = pts.reshape((-1,1,2))    
        
    img = cv2.polylines(img, [pts], True, color, thickness=weight)
    return img


def fill_poly_on_image(img, coordinates, color):
    
    pts = np.array(coordinates, np.int32)
    pts = pts.reshape((-1,1,2))
    img = cv2.fillPoly(img, [pts], color)
    return img


def draw_line_on_image(img, x1, y1, x2, y2, color, use_weight=True):
    
    weight = 1
    if (use_weight):
        weight = 2
        
    img = cv2.line(img, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness=weight)
    return img


def draw_circle_on_image(img, x1, y1, radius, color, use_weight=True):
    
    weight = 1
    if (use_weight):
        weight = 2
        
    img = cv2.circle(img,(int(x1),int(y1)), radius, color, thickness=weight)
    return img


def show_image(img):
    cv2.imshow('hello_world',img)

    #waitKey() waits for a key press to close the window and 0 specifies indefinite loop
    #cv2.waitKey(0)
    # cv2.destroyAllWindows() simply destroys all the windows we created.
    #cv2.destroyAllWindows()
    c = cv2.waitKey()
    if(c>=0):
        return -1
    return 0


def get_p_at_d(p,v,d):
    v2 = (v[0]*d, v[1]*d)
    pTemp = (p[0]+v2[0], p[1]+v2[1])
    return pTemp

def get_norm(v):
    vd = math.sqrt(v[0]*v[0]+v[1]*v[1])
    v_norm = (v[0]/vd, v[1]/vd)
    return v_norm

def get_dist(p1, p2):
    d = math.sqrt(pow(p2[0]-p1[0],2) + pow(p2[1]-p1[1],2))
    return d

def get_mid(p1, p2):
    return ((p1[0]+p2[0])/2.0, (p1[1]+p2[1])/2.0)

# Cleaned Functions

In [ ]:
import cv2
import math
from sympy import Point, Polygon, Line
from skimage.draw import line

def get_mask_volume(mask, K=20, is_binary_image = True):
    '''
    Computes the volume of the mask with the assumption that the 
    the the width of the segments are used as the radius of a cylinder.

    Parameters
    ----------
    mask : numpy array of 2 or 3 dimensions
           This is a binary mask which means the values contained within
           the array are 1 and 0 only.
    K : int, optional
        The number of segments(or cylinders) to divide the mask into. 
        The default is 20. 
        The greater the number the better the volume approximation, however,
        the more computationally expensive it becomes.
    is_binary_image : bool, optional
        if true, then the input mask is expected to be a binary image.
        Otherwise, it is a normal rgb image.
        Example, if you're passing in an np.array from 
        tensorflow.model.predict' for a segmentation binary classification 
        problem, then this would be a binary image or binary mask.
    use_bottom_midpoint : bool, optional
        if this is true, the segments are based on the centerline from the 
        maximum point(top of LV) to the midpoint at the bottom of the LV. 
        If False, then the centerline runs from the max point to the min point.
        The default is True.

    Returns
    -------
    None.

    '''
    
    use_bottom_midpoint = True

    poly_points = None
    midpointline = None
    minmaxline = None
    segments = None
    
    adj_mask = mask.copy()
    
    if is_binary_image:
        if(len(mask.shape) == 3):
            adj_mask = mask[:, :, 0]
    
    mask_rgb = adj_mask
    
    if is_binary_image:
        mask_rgb = get_rgb_image(adj_mask)

    #Get polyline of the mask    
    image_test1_gray = cv2.cvtColor(mask_rgb, cv2.COLOR_BGR2GRAY)
    contours, hierarchy = cv2.findContours(image_test1_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    largest_index_area = 0
    largest_area = -1
    for i in range(0, len(contours)):
        area = cv2.contourArea(contours[i])
        if (area > largest_area):
            largest_area = area
            largest_index_area = i


    #Get the shape/polyline of the largest area    
    poly_points = contours[largest_index_area]    
    
    #get the bounds
    box = cv2.boundingRect(poly_points);
    length_vert = (box[1]+box[3]) - box[1]
    length_hori = (box[0]+box[2]) - box[0]
    min_row = box[1] - length_vert
    max_row = box[1] + box[3] + length_vert
    min_col = box[0] - length_hori
    max_col = box[0] + box[2] + length_hori

    #Find the two points furthest apart
    #While looping through all points, Also find the bottom-left most point and the bottom-right most point
    index1 = -1
    index2 = -1
    dr = -999999999999.00
    d_left = 999999999999.00
    d_right = 999999999999.00
    left_index = -1
    right_index = -1
    for i in range(len(poly_points)):
        coord1 = poly_points[i][0]
        x1 = coord1[0]
        y1 = coord1[1]
        for j in range(len(poly_points)):
            if(i==j):
                continue
            coord2 = poly_points[j][0]
            x2 = coord2[0]
            y2 = coord2[1]        

            d = math.sqrt(pow(x2-x1,2) + pow(y2-y1,2))
            if(d>dr):
                dr = d
                index1 = i
                index2 = j
        
        d1 = get_dist(coord1, (min_col, max_row))
        d2 = get_dist(coord1,(max_col, max_row))
        if(d1 < d_left):
            d_left = d1
            left_index = i
        if(d2 < d_right):
            d_right = d2
            right_index = i        

    coord1 = poly_points[index1][0]
    coord2 = poly_points[index2][0]
    
    #Find min and max and get vect between them
    minP = None
    maxP = None

    if(coord1[1] > coord2[1]):
        minP = coord1
        maxP = coord2
    else:
        minP = coord2
        maxP = coord1
        
    minmaxline = [minP, maxP]
        
    #Get the midpoint
    leftMost = poly_points[left_index][0]
    rightMost = poly_points[right_index][0]
    mid_temp = get_mid(leftMost, rightMost)    
    
    #create sympy polygon object
    pts_temp = []
    for i in range(len(poly_points)):
        coord = poly_points[i][0]
        x1 = coord[0]
        y1 = coord[1]
        pt = Point(x1,y1)
        pts_temp.append(pt)
    poly = Polygon(*pts_temp)
    
    #Get vector from max point to mid point
    v1 = (maxP[0] - mid_temp[0], maxP[1] - mid_temp[1])
    vd = math.sqrt(v1[0]*v1[0]+v1[1]*v1[1])
    v1norm = (v1[0]/vd, v1[1]/vd)
    
    #Find the true midpoint     
    p1_temp = get_p_at_d(maxP, v1norm, dr/10.0)
    p2_temp = get_p_at_d(maxP, v1norm, -dr*1.1)    
    int_pts_temp = poly.intersection(Line(p1_temp, p2_temp))    
    true_mid = None
    #There should only be 2 points of intersection
    if(len(int_pts_temp) >= 2):
        if(get_dist(mid_temp, int_pts_temp[0]) < get_dist(mid_temp, int_pts_temp[1])):
            true_mid = int_pts_temp[0]
        else:
            true_mid = int_pts_temp[1]
    else:
        true_mid = mid_temp
    
    midpointline = [true_mid, maxP]
    
    startPt = maxP
    endPt = minP
    if(use_bottom_midpoint):
        endPt = true_mid
    
    #Get vector from start point to end point
    v1 = (endPt[0] - startPt[0], endPt[1] - startPt[1])
    vd = math.sqrt(v1[0]*v1[0]+v1[1]*v1[1])
    v1norm = (v1[0]/vd, v1[1]/vd)      

    #Get step size/distance    
    distance = get_dist(startPt, endPt) 
    h = distance/K    
#     d_test = int(h*float(K))    
#     true_k = K
#     if(d_test>distance):
#         true_k = true_k-1    
    true_k = K

    #Variables for the loop initialisation
    start = (startPt[0], startPt[1])
    start = get_p_at_d(start, v1norm, -h)
    vol = 0
    segments = []    
    start = startPt
    has_intersections = True
    while(has_intersections):        
        width = 0
        
        #Get point along the line starting from startPt and taking a distance of h
        #Get vect between the next point and the previous point    
        end = get_p_at_d(start, (v1norm[0], v1norm[1]), h)        
        vH = (end[0] - start[0], end[1] - start[1])
        start = (end[0], end[1])
        
        #Safety to get out of the loop
        if(get_dist(startPt, end) > distance*1.5):
            has_intersections = False
            break
        
        #Rotate vect by 90 to get the horizontal cutting line
        vPerp = (vH[1],vH[0]*-1.0)
        vPerp_norm = get_norm(vPerp)

        #Extend length of horizontal line to make sure it cuts
        phoriz1 = get_p_at_d(end, vPerp_norm, -dr/2.0)
        phoriz2 = get_p_at_d(end, vPerp_norm, dr/2.0)        
        
        p1 = Point(phoriz1[0], phoriz1[1])
        p2 = Point(phoriz2[0], phoriz2[1])
        int_pts = poly.intersection(Line(p1, p2))        
        
        if(len(int_pts) == 2):
            int1 = int_pts[0]
            int2 = int_pts[1]
            int_pt1 = (float(int1.x),float(int1.y))
            int_pt2 = (float(int2.x),float(int2.y))
            width = get_dist(int_pt1, int_pt2)            
            segments.append((int_pt1,int_pt2)) 
            
        elif (len(int_pts) > 2):                  
            
            for i in range(len(int_pts)-1):                      
                
                coord1 = int_pts[i]
                coord2 = int_pts[i+1]                
                
                int_pt1 = (float(coord1.x),float(coord1.y))
                int_pt2 = (float(coord2.x),float(coord2.y))
                width += get_dist(int_pt1, int_pt2)                
            
            #just append the first and last for now
            p1 = int_pts[0]
            p2 = int_pts[len(int_pts)-1]
            ip1 = (float(p1.x),float(p1.y))
            ip2 = (float(p2.x),float(p2.y))
            segments.append((ip1,ip2)) 
        else:
            #len(int_pts) is zero or one here
            has_intersections = False
            break
        
        rad = width/2.0
        vol += math.pi*rad*rad*h   
    
    return vol, poly_points, minmaxline, midpointline, segments



def get_intersection(mask_rgb, poly_points, line_pt1, line_pt2):
    
    temp_mask_rgb = mask_rgb.copy()    
    
    #red
    temp_mask_rgb = draw_line_on_image(temp_mask_rgb, line_pt1[0], line_pt1[1], line_pt2[0], line_pt2[1], (255,0,0), use_weight=True)
    
    #green
    temp_mask_rgb = fill_poly_on_image(temp_mask_rgb, poly_points, (0,255,0))
    
    #cyan
    temp_mask_rgb = draw_poly_on_image(temp_mask_rgb, poly_points, (255,255,0), use_weight = False)    
    
    pt1 = (int(line_pt1[0]), int(line_pt1[1]))
    pt2 = (int(line_pt2[0]), int(line_pt2[1]))
    discrete_line = list(zip(*line(*pt1, *pt2)))    
    
    int_pts = []
    prev_pix = (0,0,0)
    for i in range(len(discrete_line)):
        coord = discrete_line[i]        
        
        if(abs(coord[1])>=temp_mask_rgb.shape[0] or abs(coord[0])>=temp_mask_rgb.shape[1]):
            continue
            
        if(coord[0] <= 0):
            continue
            
        if(coord[1] <= 0):
            continue
            
        pix = temp_mask_rgb[int(coord[1]), int(coord[0])]        
        
        if np.array_equal(pix, np.asarray((255,255,0))):
            int_pts.append(coord)
        elif i > 0:
            #check for change in transition            
            if(np.array_equal(prev_pix, np.asarray((255,0,0))) and np.array_equal(pix, np.asarray((0,255,0)))):
                int_pts.append(coord)
            elif(np.array_equal(prev_pix, np.asarray((0,255,0))) and np.array_equal(pix, np.asarray((255,0,0)))):
                int_pts.append(coord)                
        
        prev_pix = pix
        
    return int_pts



def get_mask_volume_quick(mask, K=20, is_binary_image = True):
    '''
    Computes the volume of the mask with the assumption that the 
    the the width of the segments are used as the radius of a cylinder.

    Parameters
    ----------
    mask : numpy array of 2 or 3 dimensions
           This is a binary mask which means the values contained within
           the array are 1 and 0 only.
    K : int, optional
        The number of segments(or cylinders) to divide the mask into. 
        The default is 20. 
        The greater the number the better the volume approximation, however,
        the more computationally expensive it becomes.
    is_binary_image : bool, optional
        if true, then the input mask is expected to be a binary image.
        Otherwise, it is a normal rgb image.
        Example, if you're passing in an np.array from 
        tensorflow.model.predict' for a segmentation binary classification 
        problem, then this would be a binary image or binary mask.
    use_bottom_midpoint : bool, optional
        if this is true, the segments are based on the centerline from the 
        maximum point(top of LV) to the midpoint at the bottom of the LV. 
        If False, then the centerline runs from the max point to the min point.
        The default is True.

    Returns
    -------
    None.

    '''
    
    use_bottom_midpoint = True

    poly_points = None
    midpointline = None
    minmaxline = None
    segments = None
    
    adj_mask = mask.copy()
    
    if is_binary_image:
        if(len(mask.shape) == 3):
            adj_mask = mask[:, :, 0]
    
    mask_rgb = adj_mask
    
    if is_binary_image:
        mask_rgb = get_rgb_image(adj_mask)

    #Get polyline of the mask    
    image_test1_gray = cv2.cvtColor(mask_rgb, cv2.COLOR_BGR2GRAY)
    contours, hierarchy = cv2.findContours(image_test1_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    largest_index_area = 0
    largest_area = -1
    for i in range(0, len(contours)):
        area = cv2.contourArea(contours[i])
        if (area > largest_area):
            largest_area = area
            largest_index_area = i
    
    if(largest_area <= 0):
        return 0, None, None, None, None

    #Get the shape/polyline of the largest area    
    poly_points = contours[largest_index_area]    
    
    #get the bounds
    box = cv2.boundingRect(poly_points);
    length_vert = (box[1]+box[3]) - box[1]
    length_hori = (box[0]+box[2]) - box[0]
    min_row = box[1] - length_vert
    max_row = box[1] + box[3] + length_vert
    min_col = box[0] - length_hori
    max_col = box[0] + box[2] + length_hori

    #Find the two points furthest apart
    #While looping through all points, Also find the bottom-left most point and the bottom-right most point
    index1 = -1
    index2 = -1
    dr = -999999999999.00
    d_left = 999999999999.00
    d_right = 999999999999.00
    left_index = -1
    right_index = -1
    for i in range(len(poly_points)):
        coord1 = poly_points[i][0]
        x1 = coord1[0]
        y1 = coord1[1]
        for j in range(len(poly_points)):
            if(i==j):
                continue
            coord2 = poly_points[j][0]
            x2 = coord2[0]
            y2 = coord2[1]        

            d = math.sqrt(pow(x2-x1,2) + pow(y2-y1,2))
            if(d>dr):
                dr = d
                index1 = i
                index2 = j
        
        d1 = get_dist(coord1, (min_col, max_row))
        d2 = get_dist(coord1,(max_col, max_row))
        if(d1 < d_left):
            d_left = d1
            left_index = i
        if(d2 < d_right):
            d_right = d2
            right_index = i        

    coord1 = poly_points[index1][0]
    coord2 = poly_points[index2][0]
    
    #Find min and max and get vect between them
    minP = None
    maxP = None

    if(coord1[1] > coord2[1]):
        minP = coord1
        maxP = coord2
    else:
        minP = coord2
        maxP = coord1
        
    minmaxline = [minP, maxP]
        
    #Get the midpoint
    leftMost = poly_points[left_index][0]
    rightMost = poly_points[right_index][0]
    mid_temp = get_mid(leftMost, rightMost)    
    
    #create sympy polygon object
    pts_temp = []
    for i in range(len(poly_points)):
        coord = poly_points[i][0]
        x1 = coord[0]
        y1 = coord[1]
        pt = Point(x1,y1)
        pts_temp.append(pt)    
    
    #Get vector from max point to mid point
    v1 = (maxP[0] - mid_temp[0], maxP[1] - mid_temp[1])
    vd = math.sqrt(v1[0]*v1[0]+v1[1]*v1[1])
    v1norm = (v1[0]/vd, v1[1]/vd)    
    
    #Find the true midpoint     
    p1_temp = get_p_at_d(maxP, v1norm, dr)
    p2_temp = get_p_at_d(maxP, v1norm, -dr*1.10)    
    
    int_pts_temp = get_intersection(mask_rgb, poly_points, p1_temp, p2_temp)   
    pts_len = len(int_pts_temp)
    true_mid = None
    #There should only be 2 points of intersection
    if(len(int_pts_temp) >= 2):
        if(get_dist(mid_temp, int_pts_temp[0]) < get_dist(mid_temp, int_pts_temp[pts_len-1])):
            true_mid = int_pts_temp[0]
        else:
            true_mid = int_pts_temp[pts_len-1]
    else:
        true_mid = mid_temp
        
    midpointline = [true_mid, maxP]
    
    startPt = maxP
    endPt = minP
    if(use_bottom_midpoint):
        endPt = true_mid
    
    #Get vector from start point to end point
    v1 = (endPt[0] - startPt[0], endPt[1] - startPt[1])
    vd = math.sqrt(v1[0]*v1[0]+v1[1]*v1[1])
    v1norm = (v1[0]/vd, v1[1]/vd)    
    
    #Get step size/distance    
    distance = get_dist(startPt, endPt) 
    h = distance/K    
#     d_test = int(h*float(K))    
#     true_k = K
#     if(d_test>distance):
#         true_k = true_k-1    
    true_k = K

    #Variables for the loop initialisation
    start = (startPt[0], startPt[1])
    start = get_p_at_d(start, v1norm, -h)
    vol = 0
    segments = []    
    start = startPt
    has_intersections = True
    index = -1
    blank_rgb = get_rgb_image(np.zeros((mask_rgb.shape[0],mask_rgb.shape[1])))
    while(has_intersections):        
        width = 0
        index += 1
        
        #Get point along the line starting from startPt and taking a distance of h
        #Get vect between the next point and the previous point        
        end = get_p_at_d(start, (v1norm[0], v1norm[1]), h)        
        vH = (end[0] - start[0], end[1] - start[1])
        start = (end[0], end[1])
        
        #Safety to get out of the loop
        if(get_dist(startPt, end) >= dr*1.1):
            has_intersections = False
            break
        
        #Rotate vect by 90 to get the horizontal cutting line
        vPerp = (vH[1],vH[0]*-1.0)
        vPerp_norm = get_norm(vPerp)

        #Extend length of horizontal line to make sure it cuts
        phoriz1 = get_p_at_d(end, vPerp_norm, -dr)
        phoriz2 = get_p_at_d(end, vPerp_norm, dr)        
        
        p1 = phoriz1
        p2 = phoriz2    
        int_pts = get_intersection(blank_rgb, poly_points, p1, p2)        
        
        if(len(int_pts) == 2):
            int1 = int_pts[0]
            int2 = int_pts[1]
            int_pt1 = (float(int1[0]),float(int1[1]))
            int_pt2 = (float(int2[0]),float(int2[1]))
            width = get_dist(int_pt1, int_pt2)            
            segments.append((int_pt1,int_pt2)) 
            
        elif (len(int_pts) > 2):
            #just append the first and last for now
            p1 = int_pts[0]
            p2 = int_pts[len(int_pts)-1]
            int_pt1 = (float(p1[0]),float(p1[1]))
            int_pt2 = (float(p2[0]),float(p2[1]))
            width += get_dist(int_pt1, int_pt2)            
            segments.append((int_pt1,int_pt2)) 
        else:
            #len(int_pts) is zero or one here            
            has_intersections = False
            break        
        
        rad = width/2.0
        vol += math.pi*rad*rad*h   
        
    return vol, poly_points, minmaxline, midpointline, segments

def get_mask_volume_quick_pixelspace(mask, pixelspace, K=20, is_binary_image = True):
    '''
    Computes the volume of the mask with the assumption that the 
    the the width of the segments are used as the radius of a cylinder.

    Parameters
    ----------
    mask : numpy array of 2 or 3 dimensions
           This is a binary mask which means the values contained within
           the array are 1 and 0 only.
    K : int, optional
        The number of segments(or cylinders) to divide the mask into. 
        The default is 20. 
        The greater the number the better the volume approximation, however,
        the more computationally expensive it becomes.
    is_binary_image : bool, optional
        if true, then the input mask is expected to be a binary image.
        Otherwise, it is a normal rgb image.
        Example, if you're passing in an np.array from 
        tensorflow.model.predict' for a segmentation binary classification 
        problem, then this would be a binary image or binary mask.
    use_bottom_midpoint : bool, optional
        if this is true, the segments are based on the centerline from the 
        maximum point(top of LV) to the midpoint at the bottom of the LV. 
        If False, then the centerline runs from the max point to the min point.
        The default is True.

    Returns
    -------
    None.

    '''
    
    use_bottom_midpoint = True

    poly_points = None
    midpointline = None
    minmaxline = None
    segments = None
    
    adj_mask = mask.copy()
    
    if is_binary_image:
        if(len(mask.shape) == 3):
            adj_mask = mask[:, :, 0]
    
    mask_rgb = adj_mask
    
    if is_binary_image:
        mask_rgb = get_rgb_image(adj_mask)

    #Get polyline of the mask    
    image_test1_gray = cv2.cvtColor(mask_rgb, cv2.COLOR_BGR2GRAY)
    contours, hierarchy = cv2.findContours(image_test1_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    largest_index_area = 0
    largest_area = -1
    for i in range(0, len(contours)):
        area = cv2.contourArea(contours[i])
        if (area > largest_area):
            largest_area = area
            largest_index_area = i
    
    if(largest_area <= 0):
        return 0, None, None, None, None

    #Get the shape/polyline of the largest area    
    poly_points = contours[largest_index_area]    
    
    #get the bounds
    box = cv2.boundingRect(poly_points);
    length_vert = (box[1]+box[3]) - box[1]
    length_hori = (box[0]+box[2]) - box[0]
    min_row = box[1] - length_vert
    max_row = box[1] + box[3] + length_vert
    min_col = box[0] - length_hori
    max_col = box[0] + box[2] + length_hori

    #Find the two points furthest apart
    #While looping through all points, Also find the bottom-left most point and the bottom-right most point
    index1 = -1
    index2 = -1
    dr = -999999999999.00
    d_left = 999999999999.00
    d_right = 999999999999.00
    left_index = -1
    right_index = -1
    for i in range(len(poly_points)):
        coord1 = poly_points[i][0]
        x1 = coord1[0]
        y1 = coord1[1]
        for j in range(len(poly_points)):
            if(i==j):
                continue
            coord2 = poly_points[j][0]
            x2 = coord2[0]
            y2 = coord2[1]        

            d = math.sqrt(pow(x2-x1,2) + pow(y2-y1,2))
            if(d>dr):
                dr = d
                index1 = i
                index2 = j
        
        d1 = get_dist(coord1, (min_col, max_row))
        d2 = get_dist(coord1,(max_col, max_row))
        if(d1 < d_left):
            d_left = d1
            left_index = i
        if(d2 < d_right):
            d_right = d2
            right_index = i        

    coord1 = poly_points[index1][0]
    coord2 = poly_points[index2][0]
    
    #Find min and max and get vect between them
    minP = None
    maxP = None

    if(coord1[1] > coord2[1]):
        minP = coord1
        maxP = coord2
    else:
        minP = coord2
        maxP = coord1
        
    minmaxline = [minP, maxP]
        
    #Get the midpoint
    leftMost = poly_points[left_index][0]
    rightMost = poly_points[right_index][0]
    mid_temp = get_mid(leftMost, rightMost)    
    
    #create sympy polygon object
    pts_temp = []
    for i in range(len(poly_points)):
        coord = poly_points[i][0]
        x1 = coord[0]
        y1 = coord[1]
        pt = Point(x1,y1)
        pts_temp.append(pt)    
    
    #Get vector from max point to mid point
    v1 = (maxP[0] - mid_temp[0], maxP[1] - mid_temp[1])
    vd = math.sqrt(v1[0]*v1[0]+v1[1]*v1[1])
    v1norm = (v1[0]/vd, v1[1]/vd)    
    
    #Find the true midpoint     
    p1_temp = get_p_at_d(maxP, v1norm, dr)
    p2_temp = get_p_at_d(maxP, v1norm, -dr*1.10)    
    
    int_pts_temp = get_intersection(mask_rgb, poly_points, p1_temp, p2_temp)   
    pts_len = len(int_pts_temp)
    true_mid = None
    #There should only be 2 points of intersection
    if(len(int_pts_temp) >= 2):
        if(get_dist(mid_temp, int_pts_temp[0]) < get_dist(mid_temp, int_pts_temp[pts_len-1])):
            true_mid = int_pts_temp[0]
        else:
            true_mid = int_pts_temp[pts_len-1]
    else:
        true_mid = mid_temp
        
    midpointline = [true_mid, maxP]
    
    startPt = maxP
    endPt = minP
    if(use_bottom_midpoint):
        endPt = true_mid
    
    #Get vector from start point to end point
    v1 = (endPt[0] - startPt[0], endPt[1] - startPt[1])
    vd = math.sqrt(v1[0]*v1[0]+v1[1]*v1[1])
    v1norm = (v1[0]/vd, v1[1]/vd)    
    
    #Get step size/distance    
    distance = get_dist(startPt, endPt) 
    h = distance/K    
    
    distance_conv = get_dist((startPt[0] * pixelspace[0], startPt[1] * pixelspace[1]) , (endPt[0] * pixelspace[0], endPt[1] * pixelspace[1]))
    h_conv = distance/K    
    
#     d_test = int(h*float(K))    
#     true_k = K
#     if(d_test>distance):
#         true_k = true_k-1    
    true_k = K

    #Variables for the loop initialisation
    start = (startPt[0], startPt[1])
    start = get_p_at_d(start, v1norm, -h)
    vol = 0
    segments = []    
    start = startPt
    has_intersections = True
    index = -1
    blank_rgb = get_rgb_image(np.zeros((mask_rgb.shape[0],mask_rgb.shape[1])))
    while(has_intersections):        
        width = 0
        index += 1
        
        #Get point along the line starting from startPt and taking a distance of h
        #Get vect between the next point and the previous point        
        end = get_p_at_d(start, (v1norm[0], v1norm[1]), h)        
        vH = (end[0] - start[0], end[1] - start[1])
        start = (end[0], end[1])
        
        #Safety to get out of the loop
        if(get_dist(startPt, end) >= dr*1.1):
            has_intersections = False
            break
        
        #Rotate vect by 90 to get the horizontal cutting line
        vPerp = (vH[1],vH[0]*-1.0)
        vPerp_norm = get_norm(vPerp)

        #Extend length of horizontal line to make sure it cuts
        phoriz1 = get_p_at_d(end, vPerp_norm, -dr)
        phoriz2 = get_p_at_d(end, vPerp_norm, dr)        
        
        p1 = phoriz1
        p2 = phoriz2    
        int_pts = get_intersection(blank_rgb, poly_points, p1, p2)        
        
        if(len(int_pts) == 2):
            int1 = int_pts[0] 
            int2 = int_pts[1]
            int_pt1 = (float(int1[0]) * pixelspace[0] , float(int1[1]) * pixelspace[1])
            int_pt2 = (float(int2[0]) * pixelspace[0] , float(int2[1]) * pixelspace[1])
            width = get_dist(int_pt1, int_pt2)   
            
            int_pt1 = (float(int1[0]), float(int1[1]))
            int_pt2 = (float(int2[0]), float(int2[1]))
            segments.append((int_pt1,int_pt2)) 
            
        elif (len(int_pts) > 2):
            #just append the first and last for now
            p1 = int_pts[0]
            p2 = int_pts[len(int_pts)-1]
            int_pt1 = (float(p1[0]) * pixelspace[0] , float(p1[1]) * pixelspace[1])
            int_pt2 = (float(p2[0]) * pixelspace[0] , float(p2[1]) * pixelspace[1])
            width += get_dist(int_pt1, int_pt2)
            
            int_pt1 = (float(p1[0]), float(p1[1]))
            int_pt2 = (float(p2[0]), float(p2[1]))
            segments.append((int_pt1,int_pt2)) 
        else:
            #len(int_pts) is zero or one here            
            has_intersections = False
            break        
        
        rad = width/2.0
        vol += math.pi*rad*rad* h_conv   
        
    return vol, poly_points, minmaxline, midpointline, segments

def annotate_image(mask, polypoints, minmaxline, midpointline, segments, is_binary_image = True):
    
    adj_mask = mask.copy()
    
    if is_binary_image:
        if(len(mask.shape) == 3):
            adj_mask = mask[:, :, 0]
        
    mask_rgb = adj_mask
    
    if is_binary_image:
        mask_rgb = get_rgb_image(adj_mask)
    
    #Draw the shape/polyline of the largest area onto the mask
    if not polypoints is None:        
        mask_rgb = draw_poly_on_image(mask_rgb, polypoints, (255,255,0), False)
    
    #Draw the line between the two farthest points of the polyline.
    if not minmaxline is None:
        minP = minmaxline[0]
        maxP = minmaxline[1]
        mask_rgb = draw_line_on_image(mask_rgb, minP[0], minP[1], maxP[0], maxP[1], (0,255,0), False)
    
    #Draw the line between the highest point and the bottom mid-point.
    if not midpointline is None:
        midP = midpointline[0]
        maxP = midpointline[1]
        mask_rgb = draw_line_on_image(mask_rgb, midP[0], midP[1], maxP[0], maxP[1], (255,0,255), False)
    
    #Draw the lines of the segments that run accross the mask
    if not segments is None:
        for segment in segments:
            int_pt1 = segment[0]
            int_pt2 = segment[1]
            mask_rgb = draw_line_on_image(mask_rgb, int_pt1[0], int_pt1[1], int_pt2[0], int_pt2[1], (255,0,0), use_weight = False)
            mask_rgb = draw_circle_on_image(mask_rgb, int_pt1[0], int_pt1[1], 2, (0,0,255), False)
            mask_rgb = draw_circle_on_image(mask_rgb, int_pt2[0], int_pt2[1], 2, (0,0,255), False)
            
    return mask_rgb

In [ ]:
def load_np_from_file(filename, index):
    
    new_file_name = f'{filename}.pkl'
    if not(index is None):
        new_file_name = f'{filename}_{index}.pkl'    
    
    with open(new_file_name, 'rb') as filehandle:
        return pickle.load(filehandle)
    
    
def save_np_to_file(filename, index, data):
    
    new_file_name = f'{filename}.pkl'
    if not(index is None):
        new_file_name = f'{filename}_{index}.pkl'        
    
    with open(new_file_name, 'wb') as filehandle:
        pickle.dump(data, filehandle)

# MAKE PREDICTIONS

In [ ]:
train_dir = os.path.join(LOCAL_DATA_PATH, 'images/training')
test_dir = os.path.join(LOCAL_DATA_PATH, 'images/testing')
val_dir = os.path.join(LOCAL_DATA_PATH, 'images/validation')
annot_dir = os.path.join(LOCAL_DATA_PATH, 'annotations_binary')

In [ ]:
print(train_dir)
print(val_dir)
print(test_dir)

In [ ]:
#train_files = os.listdir(train_dir)
test_files = os.listdir(test_dir)
#val_files = os.listdir(val_dir)

#for i in range(0, len(train_files)):
#    filename = train_files[i]
#    train_files[i] = os.path.join(train_dir, filename)
    
for i in range(0, len(test_files)):
    filename = test_files[i]
    test_files[i] = os.path.join(test_dir, filename)
    
#for i in range(0, len(val_files)):
#    filename = val_files[i]
#    val_files[i] = os.path.join(val_dir, filename)

#random.shuffle(train_files)
#random.shuffle(test_files)
#random.shuffle(val_files)

#print(f'Training File Count: {len(train_files)}')
#print(f'Validation File Count: {len(val_files)}')
print(f'Test File Size: {len(test_files)}') 

test_files.sort()

for i in range(10):
    print(test_files[i])

In [ ]:
def get_avg_volume_of_masks(masks):
    
    num_examples = len(masks)
    all_vol = {}
    all_imgs = {}
    sum_vol = 0
    
    for i in range(num_examples):
        mask = masks[i]        
        vol, poly_points, minmaxline, midpointline, segments = get_mask_volume_quick(mask, K=50)
        img = annotate_image(mask, poly_points, None, midpointline, segments)
        sum_vol+= vol
        all_vol[i] = vol
        all_imgs[i] = img
        
    avg_vol = (sum_vol)/num_examples
    
    return avg_vol, all_vol, all_imgs

def get_avg_volume_of_masks_pixelspace(masks, pixelspaces):
    
    num_examples = len(masks)
    all_vol = {}
    all_imgs = {}
    sum_vol = 0
    
    for i in range(num_examples):
        mask = masks[i]      
        pixelspace1 = pixelspaces[i]
        vol, poly_points, minmaxline, midpointline, segments = get_mask_volume_quick_pixelspace(mask, pixelspace1, K=100)
        img = annotate_image(mask, poly_points, None, midpointline, segments)
        sum_vol+= vol
        all_vol[i] = vol
        all_imgs[i] = img
        
    avg_vol = (sum_vol)/num_examples
    
    return avg_vol, all_vol, all_imgs

def split_into_parts(volumes):
    ED_vols = {}
    ES_vols = {}
    
    c=0
    for i in range(0, len(volumes), 2):
        edv = volumes[i]
        esv = volumes[i+1]
        ED_vols[c] = edv
        ES_vols[c] = esv
        c+=1
        
    return ED_vols, ES_vols

def compute_ejection_fraction(ED_volumes, ES_volumes):
    
    num_examples = len(ED_volumes)
    all_ef = {}
    sum_ef = 0
    
    for i in range(num_examples):
        EDV = ED_volumes[i]
        ESV = ES_volumes[i]
        
        if(ESV>EDV):
            temp = ESV
            ESV = EDV
            EDV = temp
        
        stroke_volume = EDV - ESV
        ejection_fraction = 0
        if (EDV > 0):
            ejection_fraction = (stroke_volume / EDV) * 100.0
        sum_ef += ejection_fraction
        all_ef[i] = ejection_fraction
        
    avg_ef = (sum_ef)/num_examples
    
    return avg_ef, all_ef

In [ ]:
def get_overlay_image(echo_image, gt, pred):
    img_gt = gt
    img_pred= pred

    #Get polyline of the mask    
    image_test1_gray = cv2.cvtColor(img_gt, cv2.COLOR_BGR2GRAY)
    contours, hierarchy = cv2.findContours(image_test1_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    largest_index_area = 0
    largest_area = -1
    for i in range(0, len(contours)):
        area = cv2.contourArea(contours[i])
        if (area > largest_area):
            largest_area = area
            largest_index_area = i

    #Get the shape/polyline of the largest area    
    poly_points = contours[largest_index_area]

    #Get polyline of the mask    
    image_test1_gray = cv2.cvtColor(img_pred, cv2.COLOR_BGR2GRAY)
    contours, hierarchy = cv2.findContours(image_test1_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    largest_index_area = 0
    largest_area = -1
    for i in range(0, len(contours)):
        area = cv2.contourArea(contours[i])
        if (area > largest_area):
            largest_area = area
            largest_index_area = i

    #Get the shape/polyline of the largest area    
    poly_pointsp = contours[largest_index_area]   

    img8 = draw_poly_on_image_weight(echo_image, poly_points, (255,0,0), 4)
    img8p = draw_poly_on_image_weight(img8, poly_pointsp, (0,255,0), 4)

    return img8p

In [ ]:
def get_points(mask, is_binary_image = True):
    '''
    Gets the points of the predicted mask outline, as well as 
    the bottom left-most and right-most points.

    Parameters
    ----------
    mask : numpy array of 2 or 3 dimensions
           This is a binary mask which means the values contained within
           the array are 1 and 0 only.    
    is_binary_image : bool, optional
        if true, then the input mask is expected to be a binary image.
        Otherwise, it is a normal rgb image.
        Example, if you're passing in an np.array from 
        tensorflow.model.predict' for a segmentation binary classification 
        problem, then this would be a binary image or binary mask.
    use_bottom_midpoint : bool, optional
        if this is true, the segments are based on the centerline from the 
        maximum point(top of LV) to the midpoint at the bottom of the LV. 
        If False, then the centerline runs from the max point to the min point.
        The default is True.

    Returns
    -------
    None.

    '''
    
    use_bottom_midpoint = True

    poly_points = None
    midpointline = None
    minmaxline = None
    segments = None
    
    adj_mask = mask.copy()
    
    if is_binary_image:
        if(len(mask.shape) == 3):
            adj_mask = mask[:, :, 0]
    
    mask_rgb = adj_mask
    
    if is_binary_image:
        mask_rgb = get_rgb_image(adj_mask)

    #Get polyline of the mask    
    image_test1_gray = cv2.cvtColor(mask_rgb, cv2.COLOR_BGR2GRAY)
    contours, hierarchy = cv2.findContours(image_test1_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    largest_index_area = 0
    largest_area = -1
    for i in range(0, len(contours)):
        area = cv2.contourArea(contours[i])
        if (area > largest_area):
            largest_area = area
            largest_index_area = i
    
    if(largest_area <= 0):
        return 0, None, None, None, None

    #Get the shape/polyline of the largest area    
    poly_points = contours[largest_index_area]    
    
    #get the bounds
    box = cv2.boundingRect(poly_points);
    length_vert = (box[1]+box[3]) - box[1]
    length_hori = (box[0]+box[2]) - box[0]
    min_row = box[1] - length_vert
    max_row = box[1] + box[3] + length_vert
    min_col = box[0] - length_hori
    max_col = box[0] + box[2] + length_hori

    #Find the two points furthest apart
    #While looping through all points, Also find the bottom-left most point and the bottom-right most point
    index1 = -1
    index2 = -1
    dr = -999999999999.00
    d_left = 999999999999.00
    d_right = 999999999999.00
    left_index = -1
    right_index = -1
    for i in range(len(poly_points)):
        coord1 = poly_points[i][0]
        x1 = coord1[0]
        y1 = coord1[1]
        for j in range(len(poly_points)):
            if(i==j):
                continue
            coord2 = poly_points[j][0]
            x2 = coord2[0]
            y2 = coord2[1]        

            d = math.sqrt(pow(x2-x1,2) + pow(y2-y1,2))
            if(d>dr):
                dr = d
                index1 = i
                index2 = j
        
        d1 = get_dist(coord1, (min_col, max_row))
        d2 = get_dist(coord1,(max_col, max_row))
        if(d1 < d_left):
            d_left = d1
            left_index = i
        if(d2 < d_right):
            d_right = d2
            right_index = i        

    coord1 = poly_points[index1][0]
    coord2 = poly_points[index2][0]
    
    #Find min and max and get vect between them
    minP = None
    maxP = None

    if(coord1[1] > coord2[1]):
        minP = coord1
        maxP = coord2
    else:
        minP = coord2
        maxP = coord1
        
    minmaxline = [minP, maxP]
        
    #Get the midpoint
    leftMost = poly_points[left_index][0]
    rightMost = poly_points[right_index][0]
    mid_temp = get_mid(leftMost, rightMost)
    
    return poly_points, leftMost, rightMost

def get_bin_mask(trim_mask):
    bin_mask = np.zeros(trim_mask.shape)

    for i in range(trim_mask.shape[0]):
        for j in range(trim_mask.shape[1]): 
            if(trim_mask[i,j] == 255):
                bin_mask[i,j] = 1
                
    return bin_mask


In [ ]:
from tensorflow.keras.optimizers import Adam
from loss_functions import *
loss_funcs = Semantic_loss_functions()

# Variables to store results

In [ ]:
experiment_names = []
loss_names = ['BinaryCrossEntropy loss', 'Dice loss', 'BCE & Dice loss', 'Tversky loss', 'Focal Tversky loss' ]

In [ ]:
avg_exp_volume_errors = [0,0,0,0,0]
avg_exp_volumes = [0,0,0,0,0]
avg_exp_volumes_predicted = [0,0,0,0,0]
avg_exp_dice = [0,0,0,0,0]
avg_exp_dh = [0,0,0,0,0]
avg_exp_EF = [0,0,0,0,0]
avg_exp_EFp = [0,0,0,0,0]
avg_exp_EF_Err = [0,0,0,0,0]

all_vols_ED = []
all_vols_ES = []
all_vols_pred_ED = []
all_vols_pred_ES = []

## INFERENCE

In [ ]:
experiment_name = 'Echonet Tversky Loss Model tested on Echo Test Dataset'
model_path = MODEL_PATH
test_files = test_files
loss_function = loss_funcs.tversky_loss

model = keras.models.load_model(model_path, compile = False)
model.compile(optimizer=Adam(lr=LEARNING_RATE),
                  loss=loss_function,
                  metrics=['accuracy'], run_eagerly=True
                 )

all_predicted_masks, all_ground_truth_masks, all_probabilities = make_predictions(model, test_files)
avg_dH, all_dHs = get_Hausdorff_distance(all_predicted_masks, all_ground_truth_masks)
avg_dice, all_dice = get_Dice_Coefficient(all_predicted_masks, all_ground_truth_masks)    
     
print(f'Number of predicted Masks : {len(all_predicted_masks)}')
print('Completed All Predictions')

avg_vol, all_vol, all_imgs = get_avg_volume_of_masks(all_ground_truth_masks)
avg_volp, all_volp, all_imgsp = get_avg_volume_of_masks(all_predicted_masks)
print('Completed All Volume Computations')



In [ ]:
ED_vols, ES_vols = split_into_parts(all_vol)
avg_ef, all_ef = compute_ejection_fraction(ED_vols, ES_vols)

ED_volsp, ES_volsp = split_into_parts(all_volp)
avg_efp, all_efp = compute_ejection_fraction(ED_volsp, ES_volsp)

all_vols_ED.append(list(ED_vols.values()))
all_vols_ES.append(list(ES_vols.values()))
all_vols_pred_ED.append(list(ED_volsp.values()))
all_vols_pred_ES.append(list(ES_volsp.values()))

In [ ]:
print(experiment_name)
print()
#print(f'CAMMUS Training File Count: {len(train_files)}')
#print(f'CAMMUS Validation File Count: {len(val_files)}')
print(f'Echonet Test File Size: {len(test_files)}') 
print()
print(f'Average HD: {avg_dH}')
print(f'Average DICE: {avg_dice}')
print(f'Average Ground Truth Volume: {avg_vol}') 
print(f'Average Predicted Mask Volume: {avg_volp}')
print(f'Average Volume Error: {abs(avg_vol-avg_volp)}')
print(f'Average Ground Truth Ejection Fraction: {avg_ef}') 
print(f'Average Predicted Ejection Fraction: {avg_efp}')
print(f'Average Ejection Fraction Error: {abs(avg_ef-avg_efp)}')

avg_exp_volume_errors[3] = abs(avg_vol-avg_volp)
avg_exp_volumes[3] = avg_vol
avg_exp_volumes_predicted[3] = avg_volp
avg_exp_dice[3] = avg_dice
avg_exp_dh[3] = avg_dH
avg_exp_EF[3] = avg_ef
avg_exp_EFp[3] = avg_efp
avg_exp_EF_Err[3] = abs(avg_ef-avg_efp)

In [ ]:
predictions_path = 'Predictions_binaries'
predictions_gray_scale_path = 'Predictions_grayscale'
gt_gray_scale_path = 'ground_truths_grayscale'
overlay_path = 'Overlays'
if not os.path.exists(predictions_path):
    os.makedirs(predictions_path)
if not os.path.exists(predictions_gray_scale_path):
    os.makedirs(predictions_gray_scale_path)
if not os.path.exists(gt_gray_scale_path):
    os.makedirs(gt_gray_scale_path)
if not os.path.exists(overlay_path):
    os.makedirs(overlay_path)

x=-1
for i in range(len(all_imgs)):
    if(i%2==0):
        x+=1
    echo = cv2.imread(test_files[i])
    echo = cv2.resize(echo, (512,512), fx=1, fy=1)
    overlay = get_overlay_image(echo, all_imgs[i], all_imgsp[i])
    print('----------------------------------------------------------')
    print(f'Index {i}')
    print(test_files[i])
    print(f'gt mask VOL:{all_vol[i]}')
    print(f'pred mask VOL:{all_volp[i]}')
    print(f'vol ERR:{abs(all_vol[i] - all_volp[i])}')    
    
    print(f'gt mask EF:{all_ef[x]}')
    print(f'pred mask EF:{all_efp[x]}')
    print(f'EF ERR:{abs(all_ef[x] - all_efp[x])}')
    print(f'dice:{all_dice[i]}')
    print(f'hd:{all_dHs[i]}')
    display_list = [echo, all_imgs[i], all_imgsp[i]]
    display(display_list)
    print('----------------------------------------------------------')
    
    filename = os.path.basename(test_files[i])
    cv2.imwrite(os.path.join(predictions_path, filename), all_predicted_masks[i])
    image_test1_gray = get_rgb_image(all_predicted_masks[i])
    cv2.imwrite(os.path.join(predictions_gray_scale_path, filename), image_test1_gray)
    cv2.imwrite(os.path.join(gt_gray_scale_path, filename), get_rgb_image(all_ground_truth_masks[i]))
    cv2.imwrite(os.path.join(overlay_path, filename), overlay)    

In [ ]:
print(len(all_vol))
print(len(all_ef))

output_csv_name = 'output_measurements.csv'

with open(output_csv_name, 'w', newline='') as f:
    
    # create the csv writer
    writer = csv.writer(f)
    writer.writerow(['FILENAME', 'VOL GT MASK (pix)', 'EF GT MASK', 'VOL PRED MASK (pix)', 'EF PRED MASK', 'DICE', 'HD'])

    x=-1
    for i in range(len(all_imgs)):
        
        if(i%2==0):
            x+=1
        
        filename = os.path.basename(test_files[i])        
        # write a row to the csv file
        writer.writerow([filename, all_vol[i], all_ef[x], all_volp[i], all_efp[x], all_dice[i], all_dHs[i]])

print('complete')

## Display All Results

In [ ]:
from tabulate import tabulate

print('Cammus dataset (450 images split into train(350), val(50), test(50))')
print('Model trained with Cammus dataset (train and val split)')
print('Results for predictions and volume error on test set split is as follows:')

info = {'Loss Function' : loss_names,
        'Average EF Error' : avg_exp_EF_Err,
        #'Average EF' : avg_exp_EF,
        #'Average EF Pred' : avg_exp_EFp,
        #'Average vol' : avg_exp_volumes,
        #'Average vol Pred' : avg_exp_volumes_predicted}#,
        'Average Vol Error' : avg_exp_volume_errors,
        'Avg Dice Coeff' : avg_exp_dice,
        'Avg Hausdorff Dist' : avg_exp_dh}

print(tabulate(info, headers='keys', tablefmt='fancy_grid', floatfmt=".4f"))

In [ ]:
info = {'Loss Function' : loss_names,
        #'Average EF Error' : avg_exp_EF_Err,
        'Average EF' : avg_exp_EF,
        'Average EF Pred' : avg_exp_EFp,
        'Average vol' : avg_exp_volumes,
        'Average vol Pred' : avg_exp_volumes_predicted}#,
        #'Average Vol Error' : avg_exp_volume_errors,
        #'Avg Dice Coeff' : avg_exp_dice,
        #'Avg Hausdorff Dist' : avg_exp_dh}

print(tabulate(info, headers='keys', tablefmt='fancy_grid', floatfmt=".4f"))